In [2]:
from pyscf import gto, scf

# Define H4 chain geometry (linear chain)
mol = gto.M(
    atom='''
        H  0.0  0.0  0.0
        H  0.0  0.0  0.74
        H  0.0  0.0  1.48
        H  0.0  0.0  2.22
    ''',
    basis='sto-3g',
    spin=0,  # Singlet state (closed-shell)
    charge=0
)

# Perform Hartree-Fock calculation
mf = scf.RHF(mol)
mf.kernel()

# Extract MO coefficients
mo_coefficients = mf.mo_coeff
print("Molecular Orbital Coefficients:")
print(mo_coefficients)


converged SCF energy = -2.09789961208328
Molecular Orbital Coefficients:
[[ 0.23423487  0.54557077  0.90877516 -0.89164866]
 [ 0.40106731  0.3995843  -0.64292566  1.67000545]
 [ 0.40106731 -0.3995843  -0.64292566 -1.67000545]
 [ 0.23423487 -0.54557077  0.90877516  0.89164866]]


In [3]:
from pyscf import gto, scf

# Define H4 chain geometry (linear chain)
mol = gto.M(
    atom='''
        H  0.0  0.0  0.0
        H  0.0  0.0  0.74
        H  0.0  0.0  1.48
        H  0.0  0.0  2.22
    ''',
    basis='sto-3g',
    spin=0,  # Singlet state (closed-shell)
    charge=0
)

# Perform Hartree-Fock calculation
mf = scf.RHF(mol)
mf.conv_tol = 1e-12
mf.kernel()

# Extract MO coefficients
mo_coefficients = mf.mo_coeff
print("Molecular Orbital Coefficients:")
print(mo_coefficients)

converged SCF energy = -2.09789961208328
Molecular Orbital Coefficients:
[[ 0.23423487  0.54557077  0.90877516  0.89164866]
 [ 0.40106731  0.3995843  -0.64292566 -1.67000545]
 [ 0.40106731 -0.3995843  -0.64292566  1.67000545]
 [ 0.23423487 -0.54557077  0.90877516 -0.89164866]]


In [5]:
import json
import torch
import numpy as np

# Load matrix from JSON
with open('opt_basis.json', 'r') as f:
    data = json.load(f)
print(np.array(data).shape)

(10, 10)


In [8]:
from downfolding_methods_pytorch import nelec, norbs, fock_downfolding, Solve_fermionHam, perm_orca2pyscf
from pyscf import gto, scf, dft
import scipy

atoms = 'H2.xyz'
S = gto.M(
    atom=atoms,  # Atomic symbols and coordinates
    basis="cc-pVDZ"
).intor("int1e_ovlp")
sqrtS = scipy.linalg.sqrtm(S).real
perm = perm_orca2pyscf(
    atom=atoms,  # Atomic symbols and coordinates
    basis="cc-pVDZ"
)

proj = np.array(data).T
proj = perm @ proj @ perm.T
proj = sqrtS @ proj @ sqrtS

n_fold = norbs(atom=atoms,basis='cc-pVDZ')
ham = fock_downfolding(n_fold,('self-defined',-proj),False,atom=atoms, basis='cc-pVDZ')
E = Solve_fermionHam(ham.Ham_const, ham.int_1bd, ham.int_2bd, nele=nelec(atom=atoms, basis='sto-3G'), method='FCI')[0]
print(E)


-1.1633744903192427


In [9]:
from pyscf import gto, scf, fci

# 1. Define H2 molecule
mol = gto.Mole()
mol.atom = 'H 0 0 0; H 0 0 0.74'  # 0.74 angstrom ~ equilibrium distance
mol.basis = 'cc-pvdz'
mol.spin = 0  # Singlet
mol.charge = 0
mol.build()

# 2. Run Hartree-Fock
mf = scf.RHF(mol)
mf.kernel()

# 3. Run FCI
cisolver = fci.FCI(mol, mf.mo_coeff)
fci_energy, fci_vector = cisolver.kernel()

# 4. Print result
print("FCI Energy (cc-pVDZ) for H2:", fci_energy)


converged SCF energy = -1.12870009355644
FCI Energy (cc-pVDZ) for H2: -1.1633744903192413
